In [5]:
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
image = cv2.imread('D:/Learning/Python/Practise work/Newspaper-extract.png') # reading the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
(thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
# cv2.imwrite('binary.png', binary)

In [3]:
cv2.imshow('C:/Users/manre/binary.png', binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
img = Image.open('C:/Users/manre/binary.png')
a = np.array(img.getdata(), np.uint8).reshape(img.size[1], img.size[0])

In [12]:
np.array(img.getdata(),np.uint8)

array([255, 255, 255, ..., 255, 255, 255], dtype=uint8)

In [21]:
img.size

(610, 449)

In [7]:
print(a)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [15]:
def iteration(image: a, value: int) -> np.ndarray:
    """
    This method iterates over the provided image by converting 255's to 0's if the number of consecutive 255's are
    less the "value" provided
    """

    rows, cols = image.shape
    for row in range(0,rows):
        try:
            start = image[row].tolist().index(0) # to start the conversion from the 0 pixel
        except ValueError:
            start = 0 # if '0' is not present in that row

        count = start
        for col in range(start, cols):
            if image[row, col] == 0:
                if (col-count) <= value and (col-count) > 0:
                    image[row, count:col] = 0              
                count = col  
    return image 

In [19]:
def rlsa(image: a, horizontal: bool = True, vertical: bool = True, value: int = 22) -> np.ndarray:
    """
    rlsa(RUN LENGTH SMOOTHING ALGORITHM) is to extract the block-of-text or the Region-of-interest(ROI) from the
    document binary Image provided. Must pass binary image of ndarray type.
    """
   
    if isinstance(image, np.ndarray): # image must be binary of ndarray type
        value = int(value) if value>=0 else 0 # consecutive pixel position checker value to convert 255 to 0
        try:
            # RUN LENGTH SMOOTHING ALGORITHM working horizontally on the image
           
            if horizontal:
                image = iteration(image, value)  
           
            # RUN LENGTH SMOOTHING ALGORITHM working vertically on the image
            if vertical:
                value = 25
                image = image.T
                image = iteration(image, value)
                image = image.T
           
        except (AttributeError, ValueError) as e:
            image = None
            print("ERROR: ", e, "\n")
            print('Image must be an numpy ndarray and must be in "binary". Use Opencv/PIL to convert the image to binary.\n')
            print("import cv2;\nimage=cv2.imread('path_of_the_image');\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY);\n\
                (thresh, image_binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)\n")
            print("method usage --from pythonRLSA import rlsa;\nrlsa.rlsa(image_binary, True, False, 10)")
    else:
        print('Image must be an numpy ndarray and must be in binary')
        image = None
    return image


In [23]:
new_a = rlsa(a)

In [22]:
def array2PIL(arr, size):
    mode = 'L'
    arr = arr.reshape(arr.shape[0],arr.shape[1])
    if len(arr[0]) == 3:
        arr = numpy.c_[arr, 255*numpy.ones((len(arr),1), numpy.uint8)]
    return Image.frombuffer(mode, size, arr.tostring(), 'raw', mode, 0, 1)

In [32]:
Processed_image = array2PIL(new_a, img.size)

In [33]:
import pytesseract as tess
tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [36]:
# final_img = Image.open(Processed_image)
output = tess.image_to_string(Processed_image)

print(output)